## The Goal: Begin looking into other metrics, i.e., CTP-HI_low to start. 
<br>
<b>Author:</b> Meg D. Fowler 
<br>
<b>Date:</b> 1 Sept 2020 
<br><br>
<b>Short description:</b> As a place to start, let's use 12 UTC across CONUS in summer (as in Findell and Eltahir, part 2). Not perfect, but can be refined fairly easily. 
<br>

In [1]:
# Import libraries 
import comet as cm 
import numpy as np 
import xarray as xr 
import pickle
import pandas as pd
import datetime 

# Plotting utils 
import matplotlib.pyplot as plt 
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.util


In [2]:
# from numpy import f2py

## Read in data

In [2]:
# Set file names 
mainDir = '/Users/meganfowler/Documents/NCAR/Analysis/Coupling_initial/data/hrSim_CONUS/'

fileQ1 = mainDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1979_hrQ-UTCsel.nc'
fileQ2 = mainDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1980_hrQ-UTCsel.nc'
fileQ3 = mainDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1981_hrQ-UTCsel.nc'

fileT1 = mainDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1979_hrT-UTCsel.nc'
fileT2 = mainDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1980_hrT-UTCsel.nc'
fileT3 = mainDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1981_hrT-UTCsel.nc'

fileP  = mainDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1979-1981_hrP-levels-UTCsel.nc'

filePS  = mainDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1979-1981_hrPS-UTCsel.nc'
fileT2m = mainDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1979-1981_hrTREFHT-UTCsel.nc'
fileQ2m = mainDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1979-1981_hrQREFHT-UTCsel.nc'
fileSH  = mainDir+'f.e21.FHIST_BGC.f09_f09_mg17.hourlyOutput.001.cam.h1.1979-1981_hrSHFLX-UTCsel.nc'


In [3]:
# Now read in files that have profiles and concat into one DS
Q_1979 = xr.open_dataset(fileQ1, decode_times=True)
Q_1980 = xr.open_dataset(fileQ2, decode_times=True)
Q_1981 = xr.open_dataset(fileQ3, decode_times=True)
Q_full = xr.concat([Q_1979, Q_1980,Q_1981], dim="time")
print('Done with Q...')

T_1979 = xr.open_dataset(fileT1, decode_times=True)
T_1980 = xr.open_dataset(fileT2, decode_times=True)
T_1981 = xr.open_dataset(fileT3, decode_times=True)
T_full = xr.concat([T_1979, T_1980,T_1981], dim="time")
print('Done with T... ')

P_all  = xr.open_dataset(fileP,  decode_times=True)
print('Done with P...')


Done with Q...
Done with T... 
Done with P...


In [4]:
# Now read in files (all years, sfc only)
PS  = xr.open_dataset(filePS,  decode_times=True)
T2m = xr.open_dataset(fileT2m, decode_times=True)
Q2m = xr.open_dataset(fileQ2m, decode_times=True)
SH  = xr.open_dataset(fileSH, decode_times=True)


In [5]:
# Assemble into single dataset (xarray)
ds_Full = Q_full
ds_Full['Tlev'] = (('time','lev','lat','lon'), T_full.T)
ds_Full['Plev'] = (('time','lev','lat','lon'), P_all.PRESSURE)
ds_Full['PS'] = (('time','lat','lon'), PS.PS)
ds_Full['T2m'] = (('time','lat','lon'), T2m.TREFHT)
ds_Full['Q2m'] = (('time','lat','lon'), Q2m.QREFHT)
ds_Full['SHF'] = (('time','lat','lon'), SH.SHFLX)


In [9]:
ds_Full

<xarray.Dataset>
Dimensions:  (lat: 43, lev: 32, lon: 65, time: 5475)
Coordinates:
  * lat      (lat) float64 20.26 21.2 22.15 23.09 ... 57.02 57.96 58.9 59.84
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * lon      (lon) float64 220.0 221.2 222.5 223.8 ... 296.2 297.5 298.8 300.0
  * time     (time) object 1979-01-01 10:00:00 ... 1981-12-31 14:00:00
Data variables:
    Q        (time, lev, lat, lon) float32 2.7808608e-06 ... 0.0025378529
    UTC_hr   (time) float32 10.0 11.0 12.0 13.0 14.0 ... 11.0 12.0 13.0 14.0
    UTC_day  (time) float32 1.0 1.0 1.0 1.0 1.0 2.0 ... 31.0 31.0 31.0 31.0 31.0
    UTC_mon  (time) float32 1.0 1.0 1.0 1.0 1.0 1.0 ... 12.0 12.0 12.0 12.0 12.0
    UTC_yr   (time) float32 1979.0 1979.0 1979.0 1979.0 ... 1981.0 1981.0 1981.0
    Tlev     (time, lev, lat, lon) float32 250.42609 250.28642 ... 270.61188
    Plev     (time, lev, lat, lon) float32 364.34656 364.34656 ... 97498.84
    PS       (time, lat, lon) float32 102068.03 102079.78 ... 98192.0 98230.06
    T2m      (time, lat, lon) float32 293.90747 293.6864 ... 271.09384 271.51926
    Q2m      (time, lat, lon) float32 0.011486784 0.011303191 ... 0.0027535888
    SHF      (time, lat, lon) float32 39.52112 41.92458 ... 26.336369 28.1356
Attributes:
    creation_date:  Mon Aug 31 17:46:50 MDT 2020
    Conventions:    None
    source_file:    /glade/scratch/mdfowler/processedData/f.e21.FHIST_BGC.f09...
    title:          NCL used to get UTC; 4 early morning hours saved out.

## Now try Ahmed's version 

Find fortran code here: ~/opt/anaconda3/lib/python3.7/site-packages/comet/metrics/fortran

In [14]:
iLat  = 10
iLon  = 10 
iTime = 10 

DF = ds_Full.isel(lat=iLat,lon=iLon,time=iTime).to_dataframe()
DF

,lat,lon,Q,time,UTC_hr,UTC_day,UTC_mon,UTC_yr,Tlev,Plev,PS,T2m,Q2m,SHF
lev,,,,,,,,,,,,,,
3.643466,29.685864,232.5,0.000003,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,238.908295,364.346558,102915.773438,288.028412,0.008617,44.088383
7.594820,29.685864,232.5,0.000003,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,229.048523,759.481995,102915.773438,288.028412,0.008617,44.088383
14.356632,29.685864,232.5,0.000003,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,220.940231,1435.663208,102915.773438,288.028412,0.008617,44.088383
24.612220,29.685864,232.5,0.000003,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,215.637268,2461.221924,102915.773438,288.028412,0.008617,44.088383
35.923250,29.685864,232.5,0.000003,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,211.150925,3592.324951,102915.773438,288.028412,0.008617,44.088383
43.193750,29.685864,232.5,0.000003,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,209.189697,4319.375000,102915.773438,288.028412,0.008617,44.088383
51.677499,29.685864,232.5,0.000003,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,207.920319,5167.750000,102915.773438,288.028412,0.008617,44.088383
61.520498,29.685864,232.5,0.000003,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,207.280960,6152.049805,102915.773438,288.028412,0.008617,44.088383
73.750958,29.685864,232.5,0.000003,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,206.494720,7375.095703,102915.773438,288.028412,0.008617,44.088383


In [41]:
DF.reindex(index=DF.index[::-1])

,lat,lon,Q,time,UTC_hr,UTC_day,UTC_mon,UTC_yr,Tlev,Plev,PS,T2m,Q2m,SHF
lev,,,,,,,,,,,,,,
992.556095,29.685864,232.5,0.008011,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,286.987427,102149.679688,102915.773438,288.028412,0.008617,44.088383
976.325407,29.685864,232.5,0.007749,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,285.500671,100473.484375,102915.773438,288.028412,0.008617,44.088383
957.485480,29.685864,232.5,0.007609,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,283.960327,98522.117188,102915.773438,288.028412,0.008617,44.088383
936.198398,29.685864,232.5,0.006129,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,282.656494,96317.289062,102915.773438,288.028412,0.008617,44.088383
912.644547,29.685864,232.5,0.001331,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,284.106262,93877.679688,102915.773438,288.028412,0.008617,44.088383
887.020249,29.685864,232.5,0.001057,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,284.090881,91223.617188,102915.773438,288.028412,0.008617,44.088383
859.534767,29.685864,232.5,0.000927,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,283.329773,88376.781250,102915.773438,288.028412,0.008617,44.088383
820.858369,29.685864,232.5,0.001117,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,282.855835,84370.843750,102915.773438,288.028412,0.008617,44.088383
763.404481,29.685864,232.5,0.002614,1979-01-03 10:00:00,10.0,3.0,1.0,1979.0,282.616180,78420.000000,102915.773438,288.028412,0.008617,44.088383


In [42]:
#--------------------------------
# Compute CTP-HiLow
#--------------------------------
from comet.metrics.fortran import ctp

DFtest = DF.reindex(index=DF.index[::-1])

ctp,hilow = ctp.conv_trig_pot_mod.ctp_hi_low(DFtest['Tlev'].values,
                                             DFtest['Q'].values,
                                             DFtest['Plev'].values,
                                             DFtest['T2m'].values,
                                             DFtest['Q2m'].values,
                                             DFtest['PS'].values,miss)


In [43]:
ctp

-257.36322021484375

In [44]:
hilow

30.979354858398438

## Make my own version in python?

In [21]:
# Choose indices for lat and lon: 
iLat = 10
iLon = 10 

# Isolate just this point in dataset 
ds_Sel = ds_Full.isel(lat=iLat, lon=iLon)

print('Lat/lon choice from full array: ')
print('lat: %.2f  , lon:  %.2f' % (ds_Full.lat[iLat].values,ds_Full.lon[iLon].values))
print()
print('Confirm this was chosen: ')
print('lat: %.2f  , lon:  %.2f' % (ds_Sel.lat.values,ds_Sel.lon.values))


Lat/lon choice from full array: 
lat: 29.69  , lon:  232.50

Confirm this was chosen: 
lat: 29.69  , lon:  232.50


In [22]:
# Define length of dimensions 
nLev  = len(ds_Sel.lev)
nTime = len(ds_Sel.time)


In [5]:
# # Test sounding
# miss = -9999.
# ds   = pd.read_csv('/Users/meganfowler/Documents/NCAR/Analysis/Coupling_initial/test/SOUNDING0.csv', sep=' ').astype(float)


In [23]:
# Define constants from fortran script
Rd    = 287.04 
cp    = 1005.7
R_cp  = Rd/cp
C2K   = 273.15
Lv    = 2.5e6
Lv_cp = Lv/cp
Rv    = 461.5 
grav  = 9.81
ep    = 0.622

# Constants for dewpoint temperature routine
A = 610.8
B = 237.3
C = 17.2693882

# Constants for saturation specific humidity 
t0     = 273.15
ep     = 0.622 
es0    = 6.11 
a      = 17.269
b      = 35.86
onemep = 1.0 - ep


# Number of segments to compute calculation over from 100mb to 300mb above ground
nsegments = 20


In [24]:
# Ensure pressure levels aren't  higher than the surface pressure 
iProblem = np.where(ds_Sel.Plev[:,-1] > ds_Sel.PS)[0]  # Indices where profile > PS 

if len(iProblem)>0:
    print('***** ERROR: lowest pressure level > surface pressure *****')
else:
    print('Looks good here! Surface pressure test is passed successfully.')


Looks good here! Surface pressure test is passed successfully.


In [36]:
# Get 50, 100, 150, and 300mb levels above the ground
p50   =   ds_Sel.PS.values - 5000.0
p100  =   ds_Sel.PS.values - 10000.0
p150  =   ds_Sel.PS.values - 15000.0
p300  =   ds_Sel.PS.values - 30000.0


In [47]:
# Perform everything per day 

#for iT in range(nTime):

iT = 10
# ---- Indent should start here 

# Get index above and below each of the desired pressure levels (50,100,150,300)
i50  = np.where(ds_Sel.Plev.values[iT,:] - p50[iT] >= 0)[0]
lo50 = i50[0]
up50 = i50[0]-1

i100  = np.where(ds_Sel.Plev.values[iT,:] - p100[iT] >= 0)[0]
lo100 = i100[0]
up100 = i100[0]-1

i150  = np.where(ds_Sel.Plev.values[iT,:] - p150[iT] >= 0)[0]
lo150 = i150[0]
up150 = i150[0]-1

i300   = np.where(ds_Sel.Plev.values[iT,:] - p300[iT] >= 0)[0]
lo300 = i300[0]
up300 = i300[0]-1

# ===========================================================

# ######## Low-level humidity section ######### 

# From fortran code: "Perform linear interpolation to extract each value at desired level. 
# This is done by finding the y-intercept equal to the pressure level minus the desired 
# level (basically find the temp and dew point corresponding to the level)

x_up    =   ds_Sel.Plev.values[iT,up50]-p50[iT]
x_lo    =   ds_Sel.Plev.values[iT, lo50]-p50[iT]
y_up    =   ds_Sel.Tlev.values[iT, up50]
y_lo    =   ds_Sel.Tlev.values[iT,lo50]
temp50  =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

y_up    =   ds_Sel.Q.values[iT, up50]
y_lo    =   ds_Sel.Q.values[iT, lo50]
qhum50  =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

x_up    =   ds_Sel.Plev.values[iT, up150]-p150[iT]
x_lo    =   ds_Sel.Plev.values[iT, lo150]-p150[iT]
y_up    =   ds_Sel.Tlev.values[iT, up150]
y_lo    =   ds_Sel.Tlev.values[iT, lo150]
temp150 =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

y_up    =   ds_Sel.Q.values[iT, up150]
y_lo    =   ds_Sel.Q.values[iT, lo150]
qhum150 =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up


#----- Calculate dew point temperature [K] -----
# dew_point( qhum50 , p50 , tdew50 , missing )

e50    =  (qhum50*(p50/1e2))/(0.622+0.378*qhum50)  
e50    =  e50*1e2
tdew50 =  ( (np.log(e50/A)*B) / (C-np.log(e50/A)) ) + 273.15

e150    =  (qhum150*(p150/1e2))/(0.622+0.378*qhum150)  
e150    =  e150*1e2
tdew150 =  ( (np.log(e150/A)*B) / (C-np.log(e150/A)) ) + 273.15

#----- Compute Hi_low variable ------ 
HI_low = (temp50-tdew50) + (temp150 - tdew150)

# ===========================================================

# ######## Convective Triggering Potential section ######### 

# Interpolation as above 
x_up    =   ds_Sel.Plev.values[iT, up100]-p100[iT]
x_lo    =   ds_Sel.Plev.values[iT, lo100]-p100[iT]
y_up    =   ds_Sel.Tlev.values[iT, up100]
y_lo    =   ds_Sel.Tlev.values[iT, lo100]
temp100 =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

y_up    =   ds_Sel.Q.values[iT, up100]
y_lo    =   ds_Sel.Q.values[iT, lo100]
qhum100 =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

x_up    =   ds_Sel.Plev.values[iT, up300]-p300[iT]
x_lo    =   ds_Sel.Plev.values[iT, lo300]-p300[iT]
y_up    =   ds_Sel.Tlev.values[iT, up300]
y_lo    =   ds_Sel.Tlev.values[iT, lo300]
temp300 =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

y_up    =   ds_Sel.Q.values[iT, up300]
y_lo    =   ds_Sel.Q.values[iT, lo300]
qhum300 =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

#------ Chop up integration into 20 segments from 100mb to 300 mb above ground -----
p_increment = (p100[iT]-p300[iT])/nsegments 
p_old       = p100[iT]
tseg_old    = temp100
tpar_old    =  temp100
qseg_old    =  qhum100
CTP         =  0.

for nn in range(nsegments+1): 
    # Pressure increment between defined levels (Pa)
    p_segment = p100[iT] - (p_increment * nn)
    iMatch    = np.where(ds_Sel.Plev[iT,:] > p_segment)[0]
    ilo       = iMatch[0]
    iup       = iMatch[0]-1
    
    # Perform another linear interpolation to get T and Q at the increment 
    x_up    =   ds_Sel.Plev.values[iT, iup] - p_segment
    x_lo    =   ds_Sel.Plev.values[iT, ilo] - p_segment
    y_up    =   ds_Sel.Tlev.values[iT, iup]
    y_lo    =   ds_Sel.Tlev.values[iT, ilo]
    t_segment =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

    y_up    =   ds_Sel.Q.values[iT, iup]
    y_lo    =   ds_Sel.Q.values[iT, ilo]
    q_segment =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up
                            
    # Get moist adiabatic laps rate (K/m) and depth of layer from lower to upper level
    pmid  =  ( (p_segment*np.log(p_segment)  +  p_old   *np.log(p_old))  /  np.log(p_segment*p_old) )
    tmid  =  ( (t_segment*np.log(p_segment)  +  tseg_old*np.log(p_old))  /  np.log(p_segment*p_old) )
    qmid  =  ( (q_segment*np.log(p_segment)  +  qseg_old*np.log(p_old))  /  np.log(p_segment*p_old) )
    dz    =  (p_old-p_segment) / (grav * pmid /(Rd*tmid*((1.+(qmid/ep)) / (1. + qmid))))
    
    # (Compute saturation specific humidity)
    press = pmid/1e2    # Convert to hPa 
    numerator   = ep * (es0*math.exp((a*( tmid-t0))/( tmid-b)))
    denomenator = press-onemep*(es0*math.exp((a*( tmid-t0))/( tmid-b)))
    esat  =  numerator/denomenator 
    qsat  = esat / (1 + esat)
    
    moist_lapse  =  (grav/cp) * ( (1. + (Lv    * qsat)/(   Rd*tmid   )) / (1. + (Lv**2 * qsat)/(cp*Rv*tmid**2)) )
    
    # Get parcel temperature 
    tpar = tpar_old - moist_lapse*dz 
    
    # Get mid-point temps from environment and parcel
    tpar_mid = 0.5 * (tpar + tpar_old)
    tseg_mid = 0.5 * (t_segment + tseg_old)
    
    # Integrate from old increment to increment level
    CTP  =  CTP  +  (Rd * (tpar_mid-tseg_mid) * np.log(p_old/p_segment))
    
    # Update last increment values 
    tpar_old  =  tpar
    tseg_old  =  t_segment
    qseg_old  =  q_segment
    p_old     =  p_segment
    


In [48]:
CTP

-257.3651244037676

In [40]:
HI_low[iT]

30.979355

In [46]:
print ('Results from fortran code: ')
print('CTP:    ', ctp)
print('hi_low: ', hilow)

Results from fortran code: 
CTP:     -257.36322021484375
hi_low:  30.979354858398438


## My standalone .py script: 

In [6]:
from CTP_hilow import CTP_hilow 


In [7]:
iLat  = 10
iLon  = 10 
iTime = 10 

DF = ds_Full.isel(lat=iLat,lon=iLon,time=iTime).to_dataframe()
DF = DF.reindex(index=DF.index[::-1])


In [8]:
#            CTP_hilow(Tlev, Qlev, Plev, Tsfc, Qsfc, Psfc)
CTP, HIlow = CTP_hilow(DF['Tlev'].values, DF['Q'].values, DF['Plev'].values,DF['T2m'].values[0], 
                       DF['Q2m'].values[0], DF['PS'].values[0])

Looks good here! Surface pressure test is passed successfully.


In [12]:
print(CTP, HIlow)

-257.36441024842895 30.979322001225484


In [11]:
# For comparison, orginal fortran code:

from comet.metrics.fortran import ctp

miss = np.nan
ctp,hilow = ctp.conv_trig_pot_mod.ctp_hi_low(DF['Tlev'].values,
                                             DF['Q'].values,
                                             DF['Plev'].values,
                                             DF['T2m'].values,
                                             DF['Q2m'].values,
                                             DF['PS'].values,miss)

print (ctp, hilow)

-257.36322021484375 30.979354858398438


### What about looping over time? 


In [9]:
from CTP_hilow import CTP_hilow 
import time 

In [14]:
iLat  = 10
iLon  = 10 

nT    = len(ds_Full.time)
CTP_t = np.full([nT], np.nan)
HIlow_t  = np.full([nT], np.nan)

t = time.time()

for iT in range(nT): 
    DF = ds_Full.isel(lat=iLat,lon=iLon,time=iT).to_dataframe()
    DF = DF.reindex(index=DF.index[::-1])
    
    CTP_t[iT], HIlow_t[iT] = CTP_hilow(DF['Tlev'].values, DF['Q'].values, DF['Plev'].values,DF['T2m'].values[0], 
                                       DF['Q2m'].values[0], DF['PS'].values[0])
    
#     if iT%300==0: 
#         print('Done with %.2f percent of days...' % ((iT/nT)*100) )

elapsed = time.time() - t
print('Total compute time: %.2f s' % elapsed)

Total compute time: 17.63 s


**Sanity check against fortran version of code**

In [15]:
from comet.metrics.fortran import ctp

miss = np.nan

iLat  = 10
iLon  = 10 

nT    = len(ds_Full.time)
CTP_f90 = np.full([nT], np.nan)
HIlow_f90  = np.full([nT], np.nan)

t = time.time()

for iT in range(nT): 
    DF = ds_Full.isel(lat=iLat,lon=iLon,time=iT).to_dataframe()
    DF = DF.reindex(index=DF.index[::-1])
    
    
    CTP_f90[iT],HIlow_f90[iT] = ctp.conv_trig_pot_mod.ctp_hi_low(DF['Tlev'].values,
                                                 DF['Q'].values,
                                                 DF['Plev'].values,
                                                 DF['T2m'].values,
                                                 DF['Q2m'].values,
                                                 DF['PS'].values,miss)
    
#     if iT%300==0: 
#         print('Done with %.2f percent of days...' % ((iT/nT)*100) )

elapsed = time.time() - t
print('Total compute time: %.2f s' % elapsed)

Total compute time: 11.93 s


In [24]:
# What's the difference between CTP values? HI_low values? 
ctp_diff = CTP_t - CTP_f90 
hi_diff  = HIlow_t - HIlow_f90

print('Difference in CTP calculated via Python script vs. Fortran one...')
print('Mean: %.4e' % np.nanmean(ctp_diff))
print('Max:  %.4e' % np.nanmin(ctp_diff))
print()
print('Difference in HI_low calculated via Python script vs. Fortran one...')
print('Mean: %.4e' % np.nanmean(hi_diff))
print('Max:  %.4e' % np.nanmin(hi_diff))


Difference in CTP calculated via Python script vs. Fortran one...
Mean: -5.0257e-05
Max:  -6.0269e-03

Difference in HI_low calculated via Python script vs. Fortran one...
Mean: -1.0004e-05
Max:  -5.9535e-05


In [19]:
np.nanmean(hi_diff)

-1.0003871878898405e-05

### Testing section

In [35]:
# nn = 0 

# # Pressure increment between defined levels (Pa)
# p_segment = p100[iT] - (p_increment * nn)
# iMatch    = np.where(ds_Sel.Plev[iT,:] > p_segment)[0]
# ilo       = iMatch[0]
# iup       = iMatch[0]-1

# # Perform another linear interpolation to get T and Q at the increment 
# x_up    =   ds_Sel.Plev[iT, iup] - p_segment
# x_lo    =   ds_Sel.Plev[iT, ilo] - p_segment
# y_up    =   ds_Sel.Tlev[iT, iup]
# y_lo    =   ds_Sel.Tlev[iT, ilo]
# t_segment =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

# y_up    =   ds_Sel.Q[iT, iup]
# y_lo    =   ds_Sel.Q[iT, ilo]
# q_segment =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

# # Get moist adiabatic laps rate (K/m) and depth of layer from lower to upper level
# pmid  =  ( (p_segment*np.log(p_segment)  +  p_old   *np.log(p_old))  /  np.log(p_segment*p_old) )
# tmid  =  ( (t_segment*np.log(p_segment)  +  tseg_old*np.log(p_old))  /  np.log(p_segment*p_old) )
# qmid  =  ( (q_segment*np.log(p_segment)  +  qseg_old*np.log(p_old))  /  np.log(p_segment*p_old) )
# dz    =  (p_old-p_segment) / (grav * pmid /(Rd*tmid*((1.+(qmid/ep)) / (1. + qmid))))

# # Compute saturation specific humidity 
# #qsat  =  saturation_specific_humidity( pmid, tmid, missing)
# press = pmid/1e2    # Convert to hPa 
# numerator   = ep * (es0*math.exp((a*( tmid-t0))/( tmid-b)))
# denominator = press-onemep*(es0*math.exp((a*( tmid-t0))/( tmid-b)))
# esat  =  numerator/denominator 
# qsat = esat / (1 + esat)
    
# moist_lapse  =  (grav/cp) * ( (1. + (Lv    * qsat)/(   Rd*tmid   )) / (1. + (Lv**2 * qsat)/(cp*Rv*tmid**2)) )


# # Get parcel temperature 
# tpar = tpar_old - moist_lapse*dz 

# # Get mid-point temps from environment and parcel
# tpar_mid = 0.5 * (tpar + tpar_old)
# tseg_mid = 0.5 * (t_segment + tseg_old)

# # Integrate from old increment to increment level
# CTP  =  CTP  +  (Rd * (tpar_mid-tseg_mid) * np.log(p_old/p_segment))

# # Update last increment values 
# tpar_old  =  tpar
# tseg_old  =  t_segment
# qseg_old  =  q_segment
# p_old     =  p_segment
    

In [19]:
# nn = 0 
# p_segment = p100[iT] - (p_increment * nn)
# print(p_segment)

# iMatch    = np.where(ds_Sel.Plev[iT,:] > p_segment)[0]
# print()
# print(ds_Sel.Plev[iT, iMatch[0]])
# print()
# print(ds_Sel.Plev[iT, iMatch[0]-1])



<xarray.DataArray 'PS' ()>
array(92915.7734375)
Coordinates:
    lat      float64 29.69
    lon      float64 232.5
    time     object 1979-01-03 10:00:00

<xarray.DataArray 'Plev' ()>
array(93877.68, dtype=float32)
Coordinates:
    lat      float64 29.69
    lev      float64 912.6
    lon      float64 232.5
    time     object 1979-01-03 10:00:00

<xarray.DataArray 'Plev' ()>
array(91223.62, dtype=float32)
Coordinates:
    lat      float64 29.69
    lev      float64 887.0
    lon      float64 232.5
    time     object 1979-01-03 10:00:00


In [52]:
# # Now get index above and below each of the desired pressure levels (50,100,150,300)
# lo50 = np.full([nTime], np.nan)
# up50 = np.full([nTime], np.nan)

# lo100 = np.full([nTime], np.nan)
# up100 = np.full([nTime], np.nan)

# lo150 = np.full([nTime], np.nan)
# up150 = np.full([nTime], np.nan)

# lo300 = np.full([nTime], np.nan)
# up300 = np.full([nTime], np.nan)

# for iT in range(nTime):
#     i50      = np.where(ds_Sel.Plev[iT,:] - p50[iT] >= 0)[0]
#     lo50[iT] = i50[0]
#     up50[iT] = i50[0]-1
    
#     i100      = np.where(ds_Sel.Plev[iT,:] - p100[iT] >= 0)[0]
#     lo100[iT] = i100[0]
#     up100[iT] = i100[0]-1
    
#     i150      = np.where(ds_Sel.Plev[iT,:] - p150[iT] >= 0)[0]
#     lo150[iT] = i150[0]
#     up150[iT] = i150[0]-1
    
#     i300      = np.where(ds_Sel.Plev[iT,:] - p300[iT] >= 0)[0]
#     lo300[iT] = i300[0]
#     up300[iT] = i300[0]-1


### Low-level humidity (HI_low) section

From fortran code: <br>
      !-------------------------------------------- <br>
      !--- Perform linear interpolation to extract <br>
      !--- each value at the desired level <br>
      !--- This is done by finding the y-intercept <br>
      !--- equal to the pressure level minus the <br>
      !--- desired level (basically find the temp <br>
      !--- and dew point corresponding to the level<br>
      !--------------------------------------------<br>
      <br>

In [55]:
np.shape(ds_Sel.Plev)

(5475, 32)

In [ ]:
x_up    =   plev(up50)-p50
x_lo    =   plev(lo50)-p50
y_up    =   tlev(up50)
y_lo    =   tlev(lo50)
temp50  =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

y_up    =   qlev(up50)
y_lo    =   qlev(lo50)
qhum50  =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

x_up    =   plev(up150)-p150
x_lo    =   plev(lo150)-p150
y_up    =   tlev(up150)
y_lo    =   tlev(lo150)
temp150 =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

y_up    =   qlev(up150)
y_lo    =   qlev(lo150)
qhum150 =   y_up - ((y_up-y_lo)/(x_up-x_lo))*x_up

In [46]:
ds_Sel

<xarray.Dataset>
Dimensions:  (lev: 32, time: 5475)
Coordinates:
    lat      float64 29.69
    lon      float64 232.5
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * time     (time) object 1979-01-01 10:00:00 ... 1981-12-31 14:00:00
Data variables:
    Q        (time, lev) float32 2.914262e-06 2.871931e-06 ... 0.006992577
    UTC_hr   (time) float32 10.0 11.0 12.0 13.0 14.0 ... 11.0 12.0 13.0 14.0
    UTC_day  (time) float32 1.0 1.0 1.0 1.0 1.0 2.0 ... 31.0 31.0 31.0 31.0 31.0
    UTC_mon  (time) float32 1.0 1.0 1.0 1.0 1.0 1.0 ... 12.0 12.0 12.0 12.0 12.0
    UTC_yr   (time) float32 1979.0 1979.0 1979.0 1979.0 ... 1981.0 1981.0 1981.0
    Tlev     (time, lev) float32 241.26587 230.77605 ... 285.67206 287.08374
    Plev     (time, lev) float32 364.34656 759.482 ... 100046.555 101714.766
    PS       (time) float32 102782.2 102743.62 102712.24 ... 102450.77 102477.6
    T2m      (time) float32 287.533 287.53442 287.5563 ... 288.02286 288.0778
    Q2m      (time) float32 0.0068340832 0.0068601146 ... 0.0076404978
    SHF      (time) float32 42.59062 42.44384 42.183346 ... 34.816517 33.50967
Attributes:
    creation_date:  Mon Aug 31 17:46:50 MDT 2020
    Conventions:    None
    source_file:    /glade/scratch/mdfowler/processedData/f.e21.FHIST_BGC.f09...
    title:          NCL used to get UTC; 4 early morning hours saved out.

In [17]:
# Find the index above and below each of the desired pressure levels (50,100,150,300)
testMask_lo = ds_Full.where((ds_Full.Plev - p50 >= 0),drop=False)
# nKeeps = len(testMask_lo.lev)
# iUp = nKeeps-2
# iLo = nKeeps-1
# testMask_up = ds_Full.where((ds_Full.Plev - p50 < 0), drop=True)

# Arbitrarily defined ranges to test this... could be problematic 
testMask2 = ds_Full.where( (ds_Full.Plev-p50 > -2000 ) & (ds_Full.Plev-p50 <= 800), drop=True)


In [30]:
checkArr = testMask_lo.PS
checkArr


<xarray.DataArray 'PS' (time: 5475, lat: 43, lon: 65, lev: 32)>
array([[[[       nan,        nan,        nan, ..., 102068.03 ,
          102068.03 , 102068.03 ],
         [       nan,        nan,        nan, ..., 102079.78 ,
          102079.78 , 102079.78 ],
         [       nan,        nan,        nan, ..., 102098.38 ,
          102098.38 , 102098.38 ],
         ...,
         [       nan,        nan,        nan, ..., 101945.01 ,
          101945.01 , 101945.01 ],
         [       nan,        nan,        nan, ..., 101926.77 ,
          101926.77 , 101926.77 ],
         [       nan,        nan,        nan, ..., 101904.86 ,
          101904.86 , 101904.86 ]],

        [[       nan,        nan,        nan, ..., 102176.31 ,
          102176.31 , 102176.31 ],
         [       nan,        nan,        nan, ..., 102195.305,
          102195.305, 102195.305],
         [       nan,        nan,        nan, ..., 102211.03 ,
          102211.03 , 102211.03 ],
...
         [       nan,        nan,        nan, ...,  96514.69 ,
           96514.69 ,  96514.69 ],
         [       nan,        nan,        nan, ...,  97748.04 ,
           97748.04 ,  97748.04 ],
         [       nan,        nan,        nan, ...,  98203.78 ,
           98203.78 ,  98203.78 ]],

        [[       nan,        nan,        nan, ...,  91825.484,
           91825.484,  91825.484],
         [       nan,        nan,        nan, ...,  90580.19 ,
           90580.19 ,  90580.19 ],
         [       nan,        nan,        nan, ...,  89838.37 ,
           89838.37 ,  89838.37 ],
         ...,
         [       nan,        nan,        nan, ...,  97718.766,
           97718.766,  97718.766],
         [       nan,        nan,        nan, ...,  98192.   ,
           98192.   ,  98192.   ],
         [       nan,        nan,        nan, ...,  98230.06 ,
           98230.06 ,  98230.06 ]]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 20.26 21.2 22.15 23.09 ... 57.02 57.96 58.9 59.84
  * lon      (lon) float64 220.0 221.2 222.5 223.8 ... 296.2 297.5 298.8 300.0
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * time     (time) object 1979-01-01 10:00:00 ... 1981-12-31 14:00:00

In [18]:
testMask2


<xarray.Dataset>
Dimensions:  (lat: 43, lev: 3, lon: 65, time: 5475)
Coordinates:
  * lat      (lat) float64 20.26 21.2 22.15 23.09 ... 57.02 57.96 58.9 59.84
  * lon      (lon) float64 220.0 221.2 222.5 223.8 ... 296.2 297.5 298.8 300.0
  * lev      (lev) float64 912.6 936.2 957.5
  * time     (time) object 1979-01-01 10:00:00 ... 1981-12-31 14:00:00
Data variables:
    Q        (time, lev, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    UTC_hr   (time, lev, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    UTC_day  (time, lev, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    UTC_mon  (time, lev, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    UTC_yr   (time, lev, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    Tlev     (time, lev, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    Plev     (time, lev, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    PS       (time, lat, lon, lev) float32 nan 102068.03 ... 98230.06 nan
    T2m      (time, lat, lon, lev) float32 nan 293.90747 ... 271.51926 nan
    Q2m      (time, lat, lon, lev) float32 nan 0.011486784 ... 0.0027535888 nan
    SHF      (time, lat, lon, lev) float32 nan 39.52112 39.52112 ... 28.1356 nan
Attributes:
    creation_date:  Mon Aug 31 17:46:50 MDT 2020
    Conventions:    None
    source_file:    /glade/scratch/mdfowler/processedData/f.e21.FHIST_BGC.f09...
    title:          NCL used to get UTC; 4 early morning hours saved out.

In [20]:
testMask_lo

<xarray.Dataset>
Dimensions:  (lat: 43, lev: 32, lon: 65, time: 5475)
Coordinates:
  * lat      (lat) float64 20.26 21.2 22.15 23.09 ... 57.02 57.96 58.9 59.84
  * lon      (lon) float64 220.0 221.2 222.5 223.8 ... 296.2 297.5 298.8 300.0
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * time     (time) object 1979-01-01 10:00:00 ... 1981-12-31 14:00:00
Data variables:
    Q        (time, lev, lat, lon) float32 nan nan ... 0.0025533717 0.0025378529
    UTC_hr   (time, lev, lat, lon) float32 nan nan nan nan ... 14.0 14.0 14.0
    UTC_day  (time, lev, lat, lon) float32 nan nan nan nan ... 31.0 31.0 31.0
    UTC_mon  (time, lev, lat, lon) float32 nan nan nan nan ... 12.0 12.0 12.0
    UTC_yr   (time, lev, lat, lon) float32 nan nan nan ... 1981.0 1981.0 1981.0
    Tlev     (time, lev, lat, lon) float32 nan nan nan ... 270.21024 270.61188
    Plev     (time, lev, lat, lon) float32 nan nan nan ... 97461.07 97498.84
    PS       (time, lat, lon, lev) float32 nan nan nan ... 98230.06 98230.06
    T2m      (time, lat, lon, lev) float32 nan nan nan ... 271.51926 271.51926
    Q2m      (time, lat, lon, lev) float32 nan nan ... 0.0027535888 0.0027535888
    SHF      (time, lat, lon, lev) float32 nan nan nan ... 28.1356 28.1356
Attributes:
    creation_date:  Mon Aug 31 17:46:50 MDT 2020
    Conventions:    None
    source_file:    /glade/scratch/mdfowler/processedData/f.e21.FHIST_BGC.f09...
    title:          NCL used to get UTC; 4 early morning hours saved out.

In [20]:
testMask_lo


<xarray.Dataset>
Dimensions:  (lat: 43, lev: 32, lon: 65, time: 5475)
Coordinates:
  * lon      (lon) float64 220.0 221.2 222.5 223.8 ... 296.2 297.5 298.8 300.0
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * lat      (lat) float64 20.26 21.2 22.15 23.09 ... 57.02 57.96 58.9 59.84
  * time     (time) object 1979-01-01 10:00:00 ... 1981-12-31 14:00:00
Data variables:
    Q        (time, lev, lat, lon) float32 nan nan ... 0.0025533717 0.0025378529
    UTC_hr   (time, lev, lat, lon) float32 nan nan nan nan ... 14.0 14.0 14.0
    UTC_day  (time, lev, lat, lon) float32 nan nan nan nan ... 31.0 31.0 31.0
    UTC_mon  (time, lev, lat, lon) float32 nan nan nan nan ... 12.0 12.0 12.0
    UTC_yr   (time, lev, lat, lon) float32 nan nan nan ... 1981.0 1981.0 1981.0
    Tlev     (time, lev, lat, lon) float32 nan nan nan ... 270.21024 270.61188
    Plev     (time, lev, lat, lon) float32 nan nan nan ... 97461.07 97498.84
    PS       (time, lat, lon, lev) float32 nan nan nan ... 98230.06 98230.06
    T2m      (time, lat, lon, lev) float32 nan nan nan ... 271.51926 271.51926
    Q2m      (time, lat, lon, lev) float32 nan nan ... 0.0027535888 0.0027535888
    SHF      (time, lat, lon, lev) float32 nan nan nan ... 28.1356 28.1356
Attributes:
    creation_date:  Mon Aug 31 17:46:50 MDT 2020
    Conventions:    None
    source_file:    /glade/scratch/mdfowler/processedData/f.e21.FHIST_BGC.f09...
    title:          NCL used to get UTC; 4 early morning hours saved out.

In [19]:
print(ds_Full.Plev[10,:,20,16] - p50[10,20,16])

<xarray.DataArray (lev: 32)>
array([-81355.67  , -80960.53  , -80284.35  , -79258.8   , -78127.69  ,
       -77400.64  , -76552.266 , -75567.97  , -74344.92  , -72937.89  ,
       -71388.305 , -69565.29  , -67420.61  , -64897.508 , -62190.523 ,
       -59267.21  , -55828.086 , -51782.13  , -47022.273 , -41422.55  ,
       -34834.766 , -27711.477 , -20879.57  , -14850.953 , -10041.297 ,
        -6803.5703,  -4502.672 ,  -2357.5781,   -385.8047,   1396.2031,
         2973.3594,   4354.461 ], dtype=float32)
Coordinates:
    lat      float64 39.11
    lon      float64 240.0
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
    time     object 1979-01-03 10:00:00


In [12]:
print(p300[10,10,10])
print()
print(ds_Full.Plev[10,:,10,10])


<xarray.DataArray 'PS' ()>
array(72915.77, dtype=float32)
Coordinates:
    lat      float64 29.69
    lon      float64 232.5
    time     object 1979-01-03 10:00:00

<xarray.DataArray 'Plev' (lev: 32)>
array([   364.34656,    759.482  ,   1435.6632 ,   2461.222  ,
         3592.325  ,   4319.375  ,   5167.75   ,   6152.05   ,
         7375.0957 ,   8782.123  ,  10331.713  ,  12154.724  ,
        14299.404  ,  16822.508  ,  19848.184  ,  23465.11   ,
        27720.236  ,  32726.174  ,  38615.4    ,  45543.766  ,
        53694.633  ,  62508.066  ,  70960.98   ,  78420.     ,
        84370.84   ,  88376.78   ,  91223.62   ,  93877.68   ,
        96317.29   ,  98522.12   , 100473.484  , 102149.68   ],
      dtype=float32)
Coordinates:
    lat      float64 29.69
    lon      float64 232.5
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
    time     object 1979-01-03 10:00:00


In [61]:
# Find the index above and below each of the desired pressure levels (50,100,150,300)
ilev = np.arange(32)

i50   = np.where(plev-p50 >= 0, plev, np.nan)  # This formatting of where keeps dimensionality, fills plev with NaN if condition not true
i50_2 = np.where(plev-p50 < 0, plev, np.nan)



In [90]:
# Find index above and below each of the desired pressure levels - 50,100,150,300
lo50 = np.full([nTime,nLat,nLon], np.nan)
up50 = np.full([nTime,nLat,nLon], np.nan)

lo100 = np.full([nTime,nLat,nLon], np.nan)
up100 = np.full([nTime,nLat,nLon], np.nan)

lo150 = np.full([nTime,nLat,nLon], np.nan)
up150 = np.full([nTime,nLat,nLon], np.nan)

lo300 = np.full([nTime,nLat,nLon], np.nan)
up300 = np.full([nTime,nLat,nLon], np.nan)

# Yes, this is inefficient. But can't think of another way to do it right now... 
for iT in range(nTime):
    for iLat in range(nLat):
        for iLon in range(nLon):
            
            iSplit_50  = np.where(plev[iT,:,iLat,iLon]-p50[iT,iLat,iLon] >= 0)[0]
            lo50[iT,iLat,iLon] = np.min(iSplit_50)
            up50[iT,iLat,iLon] = np.min(iSplit_50)-1
            
            iSplit_100 = np.where(plev[iT,:,iLat,iLon]-p100[iT,iLat,iLon] >= 0)[0]
            lo100[iT,iLat,iLon] = np.min(iSplit_100)
            up100[iT,iLat,iLon] = np.min(iSplit_100)-1
            
            iSplit_150 = np.where(plev[iT,:,iLat,iLon]-p150[iT,iLat,iLon] >= 0)[0]
            lo150[iT,iLat,iLon] = np.min(iSplit_150)
            up150[iT,iLat,iLon] = np.min(iSplit_150)-1
            
            iSplit_300 = np.where(plev[iT,:,iLat,iLon]-p300[iT,iLat,iLon] >= 0)[0]
            lo300[iT,iLat,iLon] = np.min(iSplit_300)
            up300[iT,iLat,iLon] = np.min(iSplit_300)-1
            
    if (iT % 100)==0:
        print('Done with %.2f percent of days' % ((iT/nTime)*100) )
            

Done with 0.00 percent of days
Done with 1.83 percent of days


KeyboardInterrupt: 